In [11]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt 
import os

# you can choose 68_face_landmarks or 81_face_landmarks

# predictor_model = 'shape_predictor_68_face_landmarks.dat'
# detector = dlib.get_frontal_face_detector()# dilib face_detector
# predictor = dlib.shape_predictor(predictor_model)

predictor_model81 = 'shape_predictor_81_face_landmarks.dat'
predictor81 = dlib.shape_predictor(predictor_model81)

In [12]:
def getclip(test_img_path):
    img = cv2.imread(test_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # rects = how many faces in the image.
    rects = detector(img, 0)
    # faces stores full_object_detection 
    faces = dlib.full_object_detections()
    
    for i in range(len(rects)):
        faces.append(predictor(img,rects[i]))
    
    if(np.array(faces).shape[0]==1):
        face_images = dlib.get_face_chips(img, faces, size=200)
        for image in face_images:
            clip = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        return clip
    
    else: return np.array(faces)

In [13]:
# get land marks of a single image
def getlandmarks(clip): 
    img = clip
    faces = detector(img,1)
    image_landmarks = []
    if (len(faces) > 0):
        for k,d in enumerate(faces):
#             cv2.rectangle(img,(d.left(),d.top()),(d.right(),d.bottom()),(255,255,255))
            shape = predictor81(img,d)
            for i in range(81):
                image_landmarks.append([shape.part(i).x, shape.part(i).y])
#                 cv2.circle(img, (shape.part(i).x, shape.part(i).y),5,(0,255,0), -1, 8)
#                 cv2.putText(img,str(i),(shape.part(i).x,shape.part(i).y),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,2555,255))
    
    image_landmarks = np.array(image_landmarks)
    return image_landmarks

In [14]:
def get_image_hull_mask(image_shape, image_landmarks, ie_polys=None):
    # get the mask of the image
    if image_landmarks.shape[0] != 81:
        raise Exception(
            'get_image_hull_mask works only with 68 landmarks')
    int_lmrks = np.array(image_landmarks, dtype=np.int)

    #hull_mask = np.zeros(image_shape[0:2]+(1,), dtype=np.float32)
    hull_mask = np.full(image_shape[0:2] + (1,), 0, dtype=np.float32)

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[0:9],
                        int_lmrks[17:18]))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[8:17],
                        int_lmrks[26:27]))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[17:20],
                        int_lmrks[8:9]))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[24:27],
                        int_lmrks[8:9]))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[19:25],
                        int_lmrks[8:9],
                        ))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[17:22],
                        int_lmrks[27:28],
                        int_lmrks[31:36],
                        int_lmrks[8:9]
                        ))), (1,))

    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[22:27],
                        int_lmrks[27:28],
                        int_lmrks[31:36],
                        int_lmrks[8:9]
                        ))), (1,))
    
    cv2.fillConvexPoly(hull_mask, cv2.convexHull(
        np.concatenate((int_lmrks[0:1],
                        int_lmrks[15:16],
                        int_lmrks[68:80]
                        ))), (1,))
    
    # nose
    cv2.fillConvexPoly(
        hull_mask, cv2.convexHull(int_lmrks[27:36]), (1,))

    if ie_polys is not None:
        ie_polys.overlay_mask(hull_mask)
    print()
    

    hull_mask = hull_mask.reshape(hull_mask.shape[0],hull_mask.shape[1])
    
    hull_mask = np.uint8(hull_mask)
    return hull_mask

In [15]:
def read_directory(directory_name,outaddress):
    # this loop is for read each image in this foder,directory_name is the foder name with images.
    for filename in os.listdir(r'./'+directory_name):
#         print(filename) #just for test
        #img is used to store the image data 
        clip = getclip(directory_name + '/' + filename)
        print(filename)
        print(clip.shape[0])
        if(clip.shape[0] == 200):
            landmarks = getlandmarks(clip)
            print(landmarks.shape)
            if(landmarks.shape[0] == 81):
                print('yes')
                mask = get_image_hull_mask(clip.shape, landmarks, ie_polys=None)
                clip = cv2.cvtColor(clip, cv2.COLOR_BGR2RGB)






                img_mask = cv2.bitwise_and(clip,clip,mask = mask)

                cv_bgr_img = cv2.cvtColor(img_mask, cv2.COLOR_RGB2BGR)

                cv2.imwrite(outaddress+'/'+filename, cv_bgr_img)




In [16]:
# put all the images you want to process in the folder "input".
directory_name = 'input'

# get the training image batch in the "output" folder. 
outaddress = 'output'
read_directory(directory_name,outaddress)

14_Aaron_Johnson_0001.jpg
200
(81, 2)
yes

14_Aaron_Johnson_0002.jpg
200
(81, 2)
yes

